# Notebook para extração dos dados de people e planets

In [2]:
! pip install pandas
! pip install requests
import pandas as pd
import requests

                                              0.0/10.6 MB ? eta -:--:--
                                              0.1/10.6 MB 1.7 MB/s eta 0:00:07
     -                                        0.3/10.6 MB 2.8 MB/s eta 0:00:04
     --                                       0.6/10.6 MB 3.8 MB/s eta 0:00:03
     --                                       0.8/10.6 MB 4.2 MB/s eta 0:00:03
     ---                                      1.0/10.6 MB 4.7 MB/s eta 0:00:03
     ----                                     1.3/10.6 MB 4.5 MB/s eta 0:00:03
     -----                                    1.5/10.6 MB 4.8 MB/s eta 0:00:02
     ------                                   1.8/10.6 MB 5.0 MB/s eta 0:00:02
     -------                                  2.0/10.6 MB 4.8 MB/s eta 0:00:02
     --------                                 2.3/10.6 MB 5.1 MB/s eta 0:00:02
     ---------                                2.6/10.6 MB 5.1 MB/s eta 0:00:02
     ----------                               2.8/10.6 MB 5

In [24]:
# Classe com métodos úteis para realizar as extrações da API
class Extracao:
    def __init__(self, nome, url):
        self.nome = nome
        self.url = url
    
    #Função que realiza o request através da url informada
    def request_unico(self):
        response = requests.get(self.url)
        if response.status_code == 200:
            dado = response.json()
            return dado
        
        else:
            print(f"Erro ao extrair dados do endpoint { self.url}. Status code: {response.status_code}")
            return False
    
    # Utilizada para realizar os requests necessários para consumir todos os dados 
    # do end point retornando em uma lista
    def request_completo(self):
        dado = self.request_unico()
        lista = dado['results']
        while dado['next']:
            self.url = dado['next']
            dado = self.request_unico()
            lista.extend(dado['results'])
  
        return lista
        
        
    #Tratamento de dados realativos às colunas que possuem listas
    def tratar_dados(self, lista):
        # Transformando as listas em strings
        try:
            for item in lista:
                for key in item:
                        if isinstance(item[key], list):
                            # Cada item da lista foi separado usando ponto e vírgula
                            item[key] = ';'.join(item[key])
        except TypeError  as e:
            print("O item não possui uma lista válida")
    
    

In [26]:
#Dados de planetas
planets = Extracao('planets', "https://swapi.dev/api/planets/" )

lista_planetas = planets.request_completo()

#Dados de pessoas
people = Extracao('people', "https://swapi.dev/api/people/" )

lista_people = people.request_completo()


In [23]:
planets.tratar_dados(lista_planetas)

In [27]:
if lista_planetas:
    df_planets = pd.DataFrame(lista_planetas)
if lista_people:
    df_people = pd.DataFrame(lista_people)

In [31]:
# Salvando os DataFrames em arquivos CSV 
df_planets.to_csv('planets.csv', index=False)
df_people.to_csv('people.csv', index=False)